## Documentación uso de Cartera

### Los import necesarios

In [5]:
import pyodbc

from Activo import *
from Accion import *
from Bono import *
from Derivado import *
from DerivadosTipos.DerivadosSCC import *

from Cartera import *

### Definimos la conexión a base de datos

In [6]:
server = '172.16.1.38'
username = 'sa'
password = 'qwerty123'
driver = '{ODBC Driver 17 for SQL Server}'
cn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + ';UID=' + username + ';PWD=' + password)

### Primero, definimos los inputs (activos) que se le entregarán a la cartera

#### Construimos una acción
 A partir de un archivo xlsx se extrae la columna 'Adj Close' para definir el historico de la acción. Hay que insertar la ruta correcta del archivo.

In [7]:
#archivo = pd.read_excel('C:\\Users\\groso\\Desktop\\Practica\\Intento\\Cartera_definitiva\\AMZN.xlsx')    
archivo = pd.read_excel('C:\\Users\\Lenovo\\Documents\\Universidad\\Practica\\Cartera_V2\\Practica\\Intento\\Cartera\\AMZN.xlsx')
columnas = ["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]
archivo = archivo[columnas][:200]
accion = pd.DataFrame()
accion['Moneda'] = ['USD']
accion['Historico'] = [[archivo["Adj Close"]]]
accion['Nombre'] = "Amazon"
accion['Inversion'] = [500000]

#### Definimos un bono

In [8]:
bono = pd.DataFrame()
bono['Riesgo'] = ['AAA']  
bono['Moneda'] = ["CLP"]
bono['TablaDesarrollo'] = ["1#25-09-2018#2,2252#0#100#2,2252|2#25-03-2019#2,2252#0#100#2,2252|3#25-09-2019#2,2252#0#100#2,2252|4#25-03-2020#2,2252#0#100#2,2252|5#25-09-2020#2,2252#0#100#2,2252|6#25-03-2021#2,2252#100#0#102,2252"]
bono['Convencion'] = ["ACT360"]
bono['FechaEmision'] = ['2018-02-20']

#### Declaramos un derivado

In [9]:
info_derivado = dict()
info_derivado["Tipo"] = 'SCC'
info_derivado["ID_Key"] = ""
info_derivado["Administradora"] = "Admin"
info_derivado["Fondo"] = "Fondo"
info_derivado["Contraparte"] = "Contraparte"
info_derivado["ID"] = 3
info_derivado["Nemotecnico"] = ""
info_derivado["Mercado"] = "Local"
fecha = datetime.date(2019, 12, 20)
hora = '1700'
info_derivado["FechaEfectiva"] = '12/11/2019'
info_derivado["FechaVenc"] = '12/11/2021'
info_derivado["AjusteFeriados"] = "CL"
info_derivado["NocionalActivo"] = 10*(10**9)
info_derivado["MonedaActivo"] = 'CLP'
info_derivado["MonedaBase"] = 'CLP'
info_derivado["TipoTasaActivo"] = 'Fija'
info_derivado["TipoTasaPasivo"] = 'Flotante'
info_derivado["TasaActivo"] = 1.45
info_derivado["TasaPasivo"] = -1000
info_derivado["FrecuenciaActivo"] = "Semi annual"
info_derivado["FrecuenciaPasivo"] = info_derivado["FrecuenciaActivo"]

info1 = pd.DataFrame([info_derivado])
derivado_info = DerivadosSCC(fecha, hora, info1, cn)
derivado = pd.DataFrame()
derivado['Derivado'] = [derivado_info]

C:\Users\Lenovo\Documents\Universidad\Practica\Cartera_V2\Practica\Intento\Cartera\DerivadosTipos\DerivadosSCC.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  info_cartera.MonedaActivo[0] = "CLP"


### La cartera

Ahora entregamos estos activos a la cartera para que esta los pueda procesar. A Cartera se le debe entregar un dataFrame con las acciones, otro para bonos y aparte para derivados, luego se declara la moneda con la que se desea evaluar los indicadores de la cartera, la fecha a valorizar y la conexión a base de datos.

In [10]:
cartera = Cartera(accion, bono, derivado, 'CLP', datetime.date(2019,2,1), cn)

Para inicializar los cálculos, se debe pedir que se seteen los historicos, retornos y volatilidades respectivas.

In [11]:
cartera.set_hist_ret_vol_totales()

#### Acciones

In [12]:
acciones = cartera.get_acciones()
acciones

In [15]:
accion = acciones[0]
accion.get_retornos()

,Amazon
0,0.000000
1,0.005883
2,-0.018282
3,-0.017160
4,0.026717
...,...
195,0.022294
196,-0.008407
197,0.002317
198,0.056923


In [16]:
accion.get_volatilidad()

,0
Amazon,0.022273


#### Bonos


In [17]:
bonos = cartera.get_bonos()
bonos

In [20]:
bono = bonos[0]
bono.get_historicos()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,CLP#3600#AAA,CLP#5400#AAA,CLP#7200#AAA,CLP#10800#AAA
0,0.981644,0.944570,0.886864,0.773673,0.584153,0.440816,0.332656,0.251038,0.142966,0.081420,0.061444,0.015039,0.003681,0.000221
1,0.981645,0.944516,0.885820,0.771612,0.582744,0.440053,0.332308,0.250946,0.143107,0.081611,0.061630,0.015136,0.003717,0.000224
2,0.981471,0.944170,0.887487,0.776731,0.587690,0.443666,0.334908,0.252814,0.144065,0.082096,0.061973,0.015192,0.003724,0.000224
3,0.981351,0.943647,0.886977,0.777696,0.590115,0.445809,0.336579,0.254093,0.144813,0.082533,0.062307,0.015279,0.003747,0.000225
4,0.981185,0.943322,0.886385,0.776282,0.587896,0.443748,0.334847,0.252669,0.143870,0.081920,0.061817,0.015124,0.003700,0.000222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.988224,0.962730,0.921999,0.839781,0.691393,0.568397,0.467241,0.384088,0.259547,0.175390,0.144178,0.054122,0.020317,0.002863
196,0.988345,0.962888,0.921000,0.835955,0.685432,0.561879,0.460603,0.377584,0.253742,0.170519,0.139785,0.051751,0.019159,0.002626
197,0.988648,0.963977,0.922294,0.837147,0.687145,0.563977,0.462894,0.379931,0.255949,0.172427,0.141524,0.052719,0.019638,0.002725
198,0.988776,0.964019,0.921452,0.836312,0.687819,0.565705,0.465276,0.382678,0.258870,0.175118,0.144031,0.054210,0.020404,0.002890


In [21]:
bono.get_volatilidad()

,0
CLP#30#AAA,0.000483
CLP#90#AAA,0.001077
CLP#180#AAA,0.001480
CLP#360#AAA,0.003011
CLP#720#AAA,0.008396
CLP#1080#AAA,0.013547
CLP#1440#AAA,0.017784
CLP#1800#AAA,0.021263
CLP#2520#AAA,0.027035
CLP#3240#AAA,0.032355


In [24]:
bono.get_correlacion()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
CLP#30#AAA,1.000000,0.720385,0.364928,0.122514,0.088938,0.096667,0.102685,0.107053,0.112772,0.115743,...,0.021915,0.039586,0.035011,0.029576,-0.001698,0.015677,0.006282,0.023702,0.024375,0.123664
CLP#90#AAA,0.720385,1.000000,0.775114,0.088764,-0.178033,-0.208293,-0.214173,-0.214387,-0.208790,-0.200305,...,0.047279,0.067183,0.063448,0.066705,0.067314,0.067851,0.083472,0.062088,0.060632,0.069017
CLP#180#AAA,0.364928,0.775114,1.000000,0.560262,0.164155,0.074028,0.038205,0.019099,-0.001680,-0.013468,...,0.080624,0.084299,0.088897,0.097648,0.114691,0.104741,0.126500,0.090706,0.088511,0.030628
CLP#360#AAA,0.122514,0.088764,0.560262,1.000000,0.869264,0.790305,0.749014,0.722428,0.682503,0.647712,...,0.073596,0.055063,0.067711,0.070718,0.065234,0.054229,0.040283,0.038453,0.038272,0.090949
CLP#720#AAA,0.088938,-0.178033,0.164155,0.869264,1.000000,0.986823,0.969510,0.952662,0.916229,0.877184,...,0.067515,0.046832,0.054019,0.052277,0.025496,0.019186,-0.014217,0.007419,0.008686,0.106104
CLP#1080#AAA,0.096667,-0.208293,0.074028,0.790305,0.986823,1.000000,0.995773,0.986632,0.958924,0.924612,...,0.066653,0.044338,0.048780,0.044589,0.012417,0.006690,-0.030731,-0.003051,-0.001400,0.110496
CLP#1440#AAA,0.102685,-0.214173,0.038205,0.749014,0.969510,0.995773,1.000000,0.997207,0.978982,0.951410,...,0.065200,0.040955,0.043330,0.037513,0.002930,-0.003378,-0.042869,-0.011727,-0.009872,0.119461
CLP#1800#AAA,0.107053,-0.214387,0.019099,0.722428,0.952662,0.986632,0.997207,1.000000,0.991252,0.971010,...,0.063277,0.037079,0.037569,0.030509,-0.005492,-0.012756,-0.053820,-0.020004,-0.018003,0.130360
CLP#2520#AAA,0.112772,-0.208790,-0.001680,0.682503,0.916229,0.958924,0.978982,0.991252,1.000000,0.994004,...,0.058710,0.029012,0.026138,0.017199,-0.020283,-0.029607,-0.072967,-0.035084,-0.032879,0.151719
CLP#3240#AAA,0.115743,-0.200305,-0.013468,0.647712,0.877184,0.924612,0.951410,0.971010,0.994004,1.000000,...,0.054088,0.021762,0.016193,0.005930,-0.032052,-0.043098,-0.087799,-0.047191,-0.044858,0.168544


#### Derivados

In [25]:
derivados = cartera.get_derivados()
derivados

In [26]:
derivado = derivados[0]
derivado.get_volatilidad()

,0
CLP#30,0.000103
CLP#90,0.000160
CLP#180,0.000225
CLP#360,0.000410
CLP#720,0.000773
CLP#1080,0.001132
CLP#1440,0.001479
CLP#1800,0.001794
CLP#2520,0.002377
CLP#3240,0.002991


In [27]:
derivado.get_correlacion()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
CLP#30#AAA,1.000000,0.720385,0.364928,0.122514,0.088938,0.096667,0.102685,0.107053,0.112772,0.115743,...,0.021915,0.039586,0.035011,0.029576,-0.001698,0.015677,0.006282,0.023702,0.024375,0.123664
CLP#90#AAA,0.720385,1.000000,0.775114,0.088764,-0.178033,-0.208293,-0.214173,-0.214387,-0.208790,-0.200305,...,0.047279,0.067183,0.063448,0.066705,0.067314,0.067851,0.083472,0.062088,0.060632,0.069017
CLP#180#AAA,0.364928,0.775114,1.000000,0.560262,0.164155,0.074028,0.038205,0.019099,-0.001680,-0.013468,...,0.080624,0.084299,0.088897,0.097648,0.114691,0.104741,0.126500,0.090706,0.088511,0.030628
CLP#360#AAA,0.122514,0.088764,0.560262,1.000000,0.869264,0.790305,0.749014,0.722428,0.682503,0.647712,...,0.073596,0.055063,0.067711,0.070718,0.065234,0.054229,0.040283,0.038453,0.038272,0.090949
CLP#720#AAA,0.088938,-0.178033,0.164155,0.869264,1.000000,0.986823,0.969510,0.952662,0.916229,0.877184,...,0.067515,0.046832,0.054019,0.052277,0.025496,0.019186,-0.014217,0.007419,0.008686,0.106104
CLP#1080#AAA,0.096667,-0.208293,0.074028,0.790305,0.986823,1.000000,0.995773,0.986632,0.958924,0.924612,...,0.066653,0.044338,0.048780,0.044589,0.012417,0.006690,-0.030731,-0.003051,-0.001400,0.110496
CLP#1440#AAA,0.102685,-0.214173,0.038205,0.749014,0.969510,0.995773,1.000000,0.997207,0.978982,0.951410,...,0.065200,0.040955,0.043330,0.037513,0.002930,-0.003378,-0.042869,-0.011727,-0.009872,0.119461
CLP#1800#AAA,0.107053,-0.214387,0.019099,0.722428,0.952662,0.986632,0.997207,1.000000,0.991252,0.971010,...,0.063277,0.037079,0.037569,0.030509,-0.005492,-0.012756,-0.053820,-0.020004,-0.018003,0.130360
CLP#2520#AAA,0.112772,-0.208790,-0.001680,0.682503,0.916229,0.958924,0.978982,0.991252,1.000000,0.994004,...,0.058710,0.029012,0.026138,0.017199,-0.020283,-0.029607,-0.072967,-0.035084,-0.032879,0.151719
CLP#3240#AAA,0.115743,-0.200305,-0.013468,0.647712,0.877184,0.924612,0.951410,0.971010,0.994004,1.000000,...,0.054088,0.021762,0.016193,0.005930,-0.032052,-0.043098,-0.087799,-0.047191,-0.044858,0.168544


Por ejemplo, si deseamos saber la volatilidad de la cartera, se debe invocar al siguiente método:

In [13]:
cartera.set_volatilidad_cartera()
volatilidad = cartera.get_volatilidad_cartera()
volatilidad

1.8575918971487535e-06

In [30]:
cartera.get_historicos_totales()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
0,0.981644,0.944570,0.886864,0.773673,0.584153,0.440816,0.332656,0.251038,0.142966,0.081420,...,0.945224,0.921186,0.893525,0.831692,0.766129,0.737445,0.617343,0.523926,0.379195,133.899994
1,0.981645,0.944516,0.885820,0.771612,0.582744,0.440053,0.332308,0.250946,0.143107,0.081611,...,0.945505,0.921717,0.894395,0.832008,0.767146,0.738167,0.618230,0.525643,0.381076,134.690002
2,0.981471,0.944170,0.887487,0.776731,0.587690,0.443666,0.334908,0.252814,0.144065,0.082096,...,0.945408,0.921966,0.894696,0.833058,0.768410,0.739573,0.620019,0.527619,0.383226,132.250000
3,0.981351,0.943647,0.886977,0.777696,0.590115,0.445809,0.336579,0.254093,0.144813,0.082533,...,0.944585,0.920531,0.892919,0.830680,0.766395,0.737442,0.616786,0.525871,0.381424,130.000000
4,0.981185,0.943322,0.886385,0.776282,0.587896,0.443748,0.334847,0.252669,0.143870,0.081920,...,0.942910,0.917479,0.888932,0.825432,0.760885,0.731264,0.610021,0.516319,0.370984,133.520004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.988224,0.962730,0.921999,0.839781,0.691393,0.568397,0.467241,0.384088,0.259547,0.175390,...,0.907636,0.874844,0.841161,0.773772,0.708368,0.676904,0.546174,0.436748,0.288212,156.479996
196,0.988345,0.962888,0.921000,0.835955,0.685432,0.561879,0.460603,0.377584,0.253742,0.170519,...,0.907997,0.875531,0.841802,0.774653,0.709478,0.678067,0.547155,0.438407,0.289882,155.169998
197,0.988648,0.963977,0.922294,0.837147,0.687145,0.563977,0.462894,0.379931,0.255949,0.172427,...,0.908136,0.875541,0.841796,0.774950,0.709788,0.677865,0.547125,0.438383,0.289857,155.529999
198,0.988776,0.964019,0.921452,0.836312,0.687819,0.565705,0.465276,0.382678,0.258870,0.175118,...,0.907870,0.875207,0.841387,0.774379,0.708851,0.677051,0.545399,0.436556,0.288046,164.639999


In [31]:
cartera.get_retornos_totales()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
0,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,3.986159e-07,-0.000057,-0.001178,-0.002667,-2.415130e-03,-0.001732,-0.001048,-0.000367,0.000990,0.002345,...,0.000296,0.000576,0.000974,0.000379,0.001326,0.000979,0.001436,0.003271,0.004949,0.005883
2,-1.767535e-04,-0.000366,0.001881,0.006612,8.451440e-03,0.008176,0.007795,0.007416,0.006669,0.005927,...,-0.000103,0.000270,0.000336,0.001262,0.001646,0.001903,0.002888,0.003751,0.005625,-0.018282
3,-1.227157e-04,-0.000554,-0.000575,0.001242,4.117493e-03,0.004818,0.004974,0.005048,0.005178,0.005311,...,-0.000870,-0.001557,-0.001988,-0.002859,-0.002626,-0.002885,-0.005228,-0.003318,-0.004712,-0.017160
4,-1.686355e-04,-0.000345,-0.000668,-0.001820,-3.766422e-03,-0.004632,-0.005158,-0.005622,-0.006535,-0.007449,...,-0.001776,-0.003322,-0.004474,-0.006337,-0.007215,-0.008413,-0.011028,-0.018331,-0.027753,0.026717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.007776e-04,0.000654,0.001633,0.002723,2.322320e-03,0.001080,-0.000238,-0.001557,-0.004192,-0.006825,...,0.000148,0.000196,0.000245,0.000711,0.000920,0.001489,0.001993,0.003151,0.004696,0.022294
196,1.226395e-04,0.000163,-0.001084,-0.004566,-8.659426e-03,-0.011534,-0.014308,-0.017078,-0.022621,-0.028166,...,0.000398,0.000784,0.000762,0.001138,0.001566,0.001717,0.001793,0.003792,0.005778,-0.008407
197,3.070018e-04,0.001131,0.001404,0.001425,2.496033e-03,0.003728,0.004961,0.006195,0.008662,0.011129,...,0.000154,0.000012,-0.000007,0.000383,0.000436,-0.000298,-0.000053,-0.000054,-0.000089,0.002317
198,1.296976e-04,0.000043,-0.000913,-0.000998,9.805628e-04,0.003059,0.005133,0.007205,0.011347,0.015487,...,-0.000293,-0.000381,-0.000487,-0.000737,-0.001320,-0.001202,-0.003161,-0.004178,-0.006267,0.056923


In [32]:
cartera.get_volatilidades_totales()

,0
CLP#30#AAA,0.000483
CLP#90#AAA,0.001077
CLP#180#AAA,0.001480
CLP#360#AAA,0.003011
CLP#720#AAA,0.008396
CLP#1080#AAA,0.013547
CLP#1440#AAA,0.017784
CLP#1800#AAA,0.021263
CLP#2520#AAA,0.027035
CLP#3240#AAA,0.032355


In [33]:
cartera.get_correlacion()

,CLP#30#AAA,CLP#90#AAA,CLP#180#AAA,CLP#360#AAA,CLP#720#AAA,CLP#1080#AAA,CLP#1440#AAA,CLP#1800#AAA,CLP#2520#AAA,CLP#3240#AAA,...,CLP#1080,CLP#1440,CLP#1800,CLP#2520,CLP#3240,CLP#3600,CLP#5400,CLP#7200,CLP#10800,Amazon
CLP#30#AAA,1.000000,0.720385,0.364928,0.122514,0.088938,0.096667,0.102685,0.107053,0.112772,0.115743,...,0.021915,0.039586,0.035011,0.029576,-0.001698,0.015677,0.006282,0.023702,0.024375,0.123664
CLP#90#AAA,0.720385,1.000000,0.775114,0.088764,-0.178033,-0.208293,-0.214173,-0.214387,-0.208790,-0.200305,...,0.047279,0.067183,0.063448,0.066705,0.067314,0.067851,0.083472,0.062088,0.060632,0.069017
CLP#180#AAA,0.364928,0.775114,1.000000,0.560262,0.164155,0.074028,0.038205,0.019099,-0.001680,-0.013468,...,0.080624,0.084299,0.088897,0.097648,0.114691,0.104741,0.126500,0.090706,0.088511,0.030628
CLP#360#AAA,0.122514,0.088764,0.560262,1.000000,0.869264,0.790305,0.749014,0.722428,0.682503,0.647712,...,0.073596,0.055063,0.067711,0.070718,0.065234,0.054229,0.040283,0.038453,0.038272,0.090949
CLP#720#AAA,0.088938,-0.178033,0.164155,0.869264,1.000000,0.986823,0.969510,0.952662,0.916229,0.877184,...,0.067515,0.046832,0.054019,0.052277,0.025496,0.019186,-0.014217,0.007419,0.008686,0.106104
CLP#1080#AAA,0.096667,-0.208293,0.074028,0.790305,0.986823,1.000000,0.995773,0.986632,0.958924,0.924612,...,0.066653,0.044338,0.048780,0.044589,0.012417,0.006690,-0.030731,-0.003051,-0.001400,0.110496
CLP#1440#AAA,0.102685,-0.214173,0.038205,0.749014,0.969510,0.995773,1.000000,0.997207,0.978982,0.951410,...,0.065200,0.040955,0.043330,0.037513,0.002930,-0.003378,-0.042869,-0.011727,-0.009872,0.119461
CLP#1800#AAA,0.107053,-0.214387,0.019099,0.722428,0.952662,0.986632,0.997207,1.000000,0.991252,0.971010,...,0.063277,0.037079,0.037569,0.030509,-0.005492,-0.012756,-0.053820,-0.020004,-0.018003,0.130360
CLP#2520#AAA,0.112772,-0.208790,-0.001680,0.682503,0.916229,0.958924,0.978982,0.991252,1.000000,0.994004,...,0.058710,0.029012,0.026138,0.017199,-0.020283,-0.029607,-0.072967,-0.035084,-0.032879,0.151719
CLP#3240#AAA,0.115743,-0.200305,-0.013468,0.647712,0.877184,0.924612,0.951410,0.971010,0.994004,1.000000,...,0.054088,0.021762,0.016193,0.005930,-0.032052,-0.043098,-0.087799,-0.047191,-0.044858,0.168544


In [34]:
cartera.get_covarianza()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,2.335714e-07,3.749771e-07,2.610239e-07,1.782663e-07,3.609039e-07,6.328819e-07,8.825871e-07,1.100081e-06,1.473443e-06,1.809865e-06,...,1.199011e-08,2.829447e-08,3.035893e-08,3.398017e-08,-2.454715e-09,2.452519e-08,1.562169e-08,7.755040e-08,1.196567e-07,1.331189e-06
1,3.749771e-07,1.160008e-06,1.235545e-06,2.878312e-07,-1.610001e-06,-3.039062e-06,-4.102368e-06,-4.909569e-06,-6.079404e-06,-6.980124e-06,...,5.764638e-08,1.070152e-07,1.226096e-07,1.707913e-07,2.168607e-07,2.365527e-07,4.625990e-07,4.527179e-07,6.633196e-07,1.655672e-06
2,2.610239e-07,1.235545e-06,2.190411e-06,2.496471e-06,2.039905e-06,1.484206e-06,1.005583e-06,6.010296e-07,-6.720624e-08,-6.449104e-07,...,1.350825e-07,1.845189e-07,2.360624e-07,3.435629e-07,5.077309e-07,5.017894e-07,9.633521e-07,9.088506e-07,1.330603e-06,1.009656e-06
3,1.782663e-07,2.878312e-07,2.496471e-06,9.064513e-06,2.197447e-05,3.223312e-05,4.010533e-05,4.624682e-05,5.555185e-05,6.309502e-05,...,2.508389e-07,2.451823e-07,3.657703e-07,5.061544e-07,5.874701e-07,5.284960e-07,6.240668e-07,7.837823e-07,1.170426e-06,6.098988e-06
4,3.609039e-07,-1.610001e-06,2.039905e-06,2.197447e-05,7.049996e-05,1.122455e-04,1.447728e-04,1.700781e-04,2.079793e-04,2.383012e-04,...,6.417488e-07,5.815507e-07,8.138019e-07,1.043484e-06,6.403492e-07,5.214643e-07,-6.142162e-07,4.217394e-07,7.408187e-07,1.984333e-05
5,6.328819e-07,-3.039062e-06,1.484206e-06,3.223312e-05,1.122455e-04,1.835146e-04,2.399030e-04,2.841881e-04,3.511890e-04,4.052623e-04,...,1.022177e-06,8.883046e-07,1.185644e-06,1.435954e-06,5.031543e-07,2.933582e-07,-2.142162e-06,-2.798107e-07,-1.926495e-07,3.334037e-05
6,8.825871e-07,-4.102368e-06,1.005583e-06,4.010533e-05,1.447728e-04,2.399030e-04,3.162862e-04,3.770862e-04,4.706916e-04,5.474561e-04,...,1.312685e-06,1.077216e-06,1.382643e-06,1.585997e-06,1.558880e-07,-1.944713e-07,-3.922956e-06,-1.412004e-06,-1.783422e-06,4.732099e-05
7,1.100081e-06,-4.909569e-06,6.010296e-07,4.624682e-05,1.700781e-04,2.841881e-04,3.770862e-04,4.520959e-04,5.697982e-04,6.680059e-04,...,1.523114e-06,1.166010e-06,1.433231e-06,1.542119e-06,-3.492933e-07,-8.779848e-07,-5.888319e-06,-2.879513e-06,-3.888233e-06,6.173729e-05
8,1.473443e-06,-6.079404e-06,-6.720624e-08,5.555185e-05,2.079793e-04,3.511890e-04,4.706916e-04,5.697982e-04,7.308748e-04,8.694626e-04,...,1.796824e-06,1.159999e-06,1.267848e-06,1.105354e-06,-1.640175e-06,-2.590938e-06,-1.015032e-05,-6.421312e-06,-9.028734e-06,9.135839e-05
9,1.809865e-06,-6.980124e-06,-6.449104e-07,6.309502e-05,2.383012e-04,4.052623e-04,5.474561e-04,6.680059e-04,8.694626e-04,1.046845e-03,...,1.981108e-06,1.041358e-06,9.400411e-07,4.561281e-07,-3.102017e-06,-4.513755e-06,-1.461726e-05,-1.033690e-05,-1.474260e-05,1.214630e-04
